-limpieza

In [38]:
#importacion de las s librerias que usaremos en este caso.
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
import re

import matplotlib.pyplot as plt
from wordcloud import WordCloud


-importacion de los datos.
    -importamos en el datafraeme'aa' referencia a 'accidentes aereos'

In [39]:

aa= pd.read_csv('AccidentesAviones.csv',encoding='utf-8')
#en alguno intentos de importacion sin usar 'utf-8' se mesclan algunas columnas.
aa.head()


,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary
0,0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
1,1,"September 07, 1909",?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
2,2,"July 12, 1912",0630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
3,3,"August 06, 1913",?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
4,4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,20,?,?,14,?,?,0,The airship flew into a thunderstorm and encou...


In [40]:
# numero de columnas y filas.
aa.shape


(5008, 18)

In [41]:
#revisemos el archivo para reconocer sus columnas y comprender que informacion nos brinda.
aa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5008 entries, 0 to 5007
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unnamed: 0              5008 non-null   int64 
 1   fecha                   5008 non-null   object
 2   HORA declarada          5008 non-null   object
 3   Ruta                    5008 non-null   object
 4   OperadOR                5008 non-null   object
 5   flight_no               5008 non-null   object
 6   route                   5007 non-null   object
 7   ac_type                 5008 non-null   object
 8   registration            5008 non-null   object
 9   cn_ln                   5008 non-null   object
 10  all_aboard              5008 non-null   object
 11  PASAJEROS A BORDO       5008 non-null   object
 12  crew_aboard             5008 non-null   object
 13  cantidad de fallecidos  5008 non-null   object
 14  passenger_fatalities    5008 non-null   object
 15  crew

-Dicionario de datos en tabla:

    -unnamed - index antiguo de la lista

    -fecha - fecha del accidente.

    -hora_declarada- hora local formato 24h hh:mm.

    -ruta - locacion del accidente.

    -operador - aerolinea en operacion de la aeronave.

    -numero de vuelo - nuemero de vuelo asignado a esta aeronave.

    -ruta_vuelo- ruta que cumplia la aeronave.

    -tipo - tipo de aeronave.

    -icao registro - registro icao de la aeronave.

    -cn_ln -serie de la aeronave .

    -all board- total personas en el vuelo.

    -pasajeros a bordo- pasajeros a bordo.

    -crew aboard -tripulantes a bordo.

    -total fatalities -total de vidas perdidas.

    -passenger fatalities - pasajeros fallecidos.

    -crew fatalities - tripulantes fallecidos.

    -ground - total de muertos en tierra

    -summary - breve descripcion del accidente y s.u posible causa.

todas las columnas estan en dtype object. lo que quiere decir que son texto o son interpretadas asi.
    

    

In [42]:
# primero asignarle un nombre entendible a las columnas.
aa.rename(columns={ 'fecha': 'fecha', 
                    'HORA declarada': 'hora',
                    'Ruta': 'pais',
                    'OperadOR': 'operadora',
                    'flight_no': 'vuelo_numero',
                    'route' : 'ruta',
                    'ac_type': 'tipo_aeronave',
                    'registration': 'registro_aeronave',
                    'cn_ln': 'registro_fuselaje',
                    'all_aboard': 'abordo',
                    'PASAJEROS A BORDO': 'pasajeros',
                    'crew_aboard': 'tripulantes',
                    'cantidad de fallecidos': 'bajas',
                    'passenger_fatalities': 'bajas_pasajeros',
                    'crew_fatalities': 'bajas_tripulacion',
                    'ground': 'bajas_tierra',
                    'summary': 'Descripcion'
}, inplace=True)
aa

,Unnamed: 0,fecha,hora,pais,operadora,vuelo_numero,ruta,tipo_aeronave,registro_aeronave,registro_fuselaje,abordo,pasajeros,tripulantes,bajas,bajas_pasajeros,bajas_tripulacion,bajas_tierra,Descripcion
0,0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
1,1,"September 07, 1909",?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
2,2,"July 12, 1912",0630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
3,3,"August 06, 1913",?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
4,4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,20,?,?,14,?,?,0,The airship flew into a thunderstorm and encou...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5003,5003,"March 28, 2021",1835,"Near Butte, Alaska",Soloy Helicopters,?,Sightseeing Charter,Eurocopter AS350B3 Ecureuil,N351SH,4598,6,5,1,5,4,1,0,The sightseeing helicopter crashed after missi...
5004,5004,"May 21, 2021",1800,"Near Kaduna, Nigeria",Military - Nigerian Air Force,?,?,Beechcraft B300 King Air 350i,NAF203,FL-891,11,7,4,11,7,4,0,"While on final approach, in poor weather condi..."
5005,5005,"June 10, 2021",0800,"Near Pyin Oo Lwin, Myanmar",Military - Myanmar Air Force,?,Naypyidaw - Anisakan,Beechcraft 1900D,4610,E-325,14,12,2,12,11,1,0,The plane was carrying military personnel and ...
5006,5006,"July 04, 2021",11:30,"Patikul, Sulu, Philippines",Military - Philippine Air Force,?,Cagayan de Oro-Lumbia - Jolo,Lockheed C-130H Hercules,5125,5125,96,88,8,50,?,?,3,"While attempting to land at Jolo Airport, the ..."


- una vez definido nuestro diccionario hay que darle un formato a cada columna.

In [43]:
# visualizamos el tipo de datos para formatear.
aa.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5008 entries, 0 to 5007
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         5008 non-null   int64 
 1   fecha              5008 non-null   object
 2   hora               5008 non-null   object
 3   pais               5008 non-null   object
 4   operadora          5008 non-null   object
 5   vuelo_numero       5008 non-null   object
 6   ruta               5007 non-null   object
 7   tipo_aeronave      5008 non-null   object
 8   registro_aeronave  5008 non-null   object
 9   registro_fuselaje  5008 non-null   object
 10  abordo             5008 non-null   object
 11  pasajeros          5008 non-null   object
 12  tripulantes        5008 non-null   object
 13  bajas              5008 non-null   object
 14  bajas_pasajeros    5008 non-null   object
 15  bajas_tripulacion  5008 non-null   object
 16  bajas_tierra       5008 non-null   object


In [44]:
# retiro las columnas 'Unnamed:' y el registro de fuselaje en este caso no los usare.
aa.drop(columns=['Unnamed: 0','registro_fuselaje'], inplace = True)
aa.columns

Index(['fecha', 'hora', 'pais', 'operadora', 'vuelo_numero', 'ruta',
       'tipo_aeronave', 'registro_aeronave', 'abordo', 'pasajeros',
       'tripulantes', 'bajas', 'bajas_pasajeros', 'bajas_tripulacion',
       'bajas_tierra', 'Descripcion'],
      dtype='object')

In [45]:
# seguimos con las mismas filas pero retiramos 2 columnas.
aa.shape

(5008, 16)

Empecemos...


In [46]:
aa['fecha']

0       September 17, 1908
1       September 07, 1909
2            July 12, 1912
3          August 06, 1913
4       September 09, 1913
               ...        
5003        March 28, 2021
5004          May 21, 2021
5005         June 10, 2021
5006         July 04, 2021
5007         July 06, 2021
Name: fecha, Length: 5008, dtype: object

In [47]:
#crear una funcion que formatee la columna fecha a dtype 'datatime64' 
#permitiendo asi usarla adecuadamente.
def convertir_fecha(texto_fecha, formato_entrada, formato_salida):
    try:
        fecha_objeto = datetime.strptime(texto_fecha, formato_entrada)
        fecha_formateada = fecha_objeto.strftime(formato_salida)
        return fecha_formateada
    except ValueError:
        return None



# Especificar los formatos de entrada y salida
formato_entrada = '%B %d, %Y'
formato_salida = '%Y-%m-%d'

# Aplicar la función de conversión a la columna 'fecha'
aa['fecha'] = aa['fecha'].apply(lambda x: convertir_fecha(x, formato_entrada, formato_salida))

# Reemplazar valores nulos por ''
aa['fecha'] = aa['fecha'].fillna('')

# Reemplazar valores '?' por un valor vacío
aa['fecha'] = aa['fecha'].str.replace('?', '')

# Convertir la columna 'fecha' al formato datetime
aa['fecha'] = pd.to_datetime(aa['fecha'], format=formato_salida)

aa['fecha'].head()

0   1908-09-17
1   1909-09-07
2   1912-07-12
3   1913-08-06
4   1913-09-09
Name: fecha, dtype: datetime64[ns]

-----------------------------------------------------------------------------

In [48]:
aa.info() # ya tenemos nuestra primer columna lista.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5008 entries, 0 to 5007
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   fecha              5008 non-null   datetime64[ns]
 1   hora               5008 non-null   object        
 2   pais               5008 non-null   object        
 3   operadora          5008 non-null   object        
 4   vuelo_numero       5008 non-null   object        
 5   ruta               5007 non-null   object        
 6   tipo_aeronave      5008 non-null   object        
 7   registro_aeronave  5008 non-null   object        
 8   abordo             5008 non-null   object        
 9   pasajeros          5008 non-null   object        
 10  tripulantes        5008 non-null   object        
 11  bajas              5008 non-null   object        
 12  bajas_pasajeros    5008 non-null   object        
 13  bajas_tripulacion  5008 non-null   object        
 14  bajas_ti

In [49]:
# formatizamos la columna 'hora'
# comencemos poniendo ':' entre los numeros 
aa['hora'] = aa['hora'].str.replace(r'(\d{2})(\d{2})', r'\1:\2', regex=True)

In [50]:
aa.shape

(5008, 16)

In [51]:
# filtramos nulos, reemplazandolo por una '' vacio.
aa['hora'] = aa['hora'].fillna('')

# convierto la hora en texto
aa['hora'] = aa['hora'].astype(str)

# Elimina los caracteres '?' de la columna
aa['hora'] = aa['hora'].str.replace('?', '')

# Elimina espacios en blanco alrededor de los valores
aa['hora'] = aa['hora'].str.strip()

# Extraemos los valores correspondientes a 'HH:MM'  
aa['hora'] = aa['hora'].str.extract(r'(\d{2}:\d{2})', expand=False)

# Rellena los valores nulos en 'hora' con una cadena vacía
aa['hora'].fillna('', inplace=True)

# Se le da formato a la hora con datetime, muestra solo la hora y convierte en string.
aa['hora'] = pd.to_datetime(aa['hora'], format='%H:%M', errors='coerce')
aa['hora'] = aa['hora'].dt.strftime('%H:%M')
aa['hora'] = aa['hora'].astype(str)


In [52]:
aa['hora']
# listo ya tenemos las dos primeras columnas.


0       17:18
1         nan
2       06:30
3         nan
4       18:30
        ...  
5003    18:35
5004    18:00
5005    08:00
5006    11:30
5007    15:00
Name: hora, Length: 5008, dtype: object

In [53]:
# la columna 'pais' dejare solo el pais donde sucedio el accidente segun la documentacion.
aa.pais.unique()

array(['Fort Myer, Virginia', 'Juvisy-sur-Orge, France',
       'Atlantic City, New Jersey', ..., 'Near Pyin Oo Lwin, Myanmar',
       'Patikul, Sulu, Philippines', 'Palana, Russia'], dtype=object)

In [54]:
aa.pais.nunique()

4125

In [55]:
aa['pais'] = aa['pais'].fillna('')
aa['pais'] = aa['pais'].str.replace('?', '')

In [56]:
# una lista con los países
lista_paises = [
    "Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda", "Argentina", "Armenia",
    "Australia", "Austria", "Azerbaijan", "Bahamas", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium",
    "Belize", "Benin", "Bhutan", "Bolivia", "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei", "Bulgaria",
    "Burkina Faso", "Burundi", "Côte d'Ivoire", "Cabo Verde", "Cambodia", "Cameroon", "Canada",
    "Central African Republic", "Chad", "Chile", "China", "Colombia", "Comoros", "Congo", "Costa Rica", "Croatia",
    "Cuba", "Cyprus", "Czech Republic", "Democratic Republic of the Congo", "Denmark", "Djibouti", "Dominica",
    "Dominican Republic", "Ecuador", "England", "Egypt", "El Salvador", "Equatorial Guinea", "Eritrea", "Estonia",
    "Eswatini", "Ethiopia", "Fiji", "Finland", "France", "Gabon", "Gambia", "Georgia", "Germany", "Ghana", "Greece",
    "Grenada", "Guatemala", "Guinea", "Guinea-Bissau", "Guyana", "Haiti", "Holy See", "Honduras", "Hungary",
    "Iceland", "India", "Indonesia", "Iran", "Iraq", "Ireland", "Israel", "Italy","Ivory Coast", "Jamaica", "Japan", "Jordan",
    "Kazakhstan", "Kenya", "Kiribati", "Kuwait", "Kyrgyzstan", "Laos", "Latvia", "Lebanon", "Lesotho", "Liberia",
    "Libya", "Liechtenstein", "Lithuania", "Luxembourg", "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali",
    "Malta", "Marshall Islands", "Mauritania", "Mauritius", "Mexico", "Micronesia", "Moldova", "Monaco", "Mongolia",
    "Montenegro", "Morocco", "Mozambique", "Myanmar", "Namibia", "Nauru", "Nepal", "Netherlands", "New Zealand",
    "Nicaragua", "Nigeria", "Niger", "North Korea", "North Macedonia", "Norway", "Oman", "Pakistan", "Palau",
    "Palestine State", "Panama", "Papua", "Papua New Guinea", "Paraguay", "Peru", "Philippines", "Poland", "Portugal", "Puerto Rico",
    "Qatar", "Romania", "Russia", "Rwanda", "Saint Kitts and Nevis", "Saint Lucia",
    "Saint Vincent and the Grenadines", "Samoa", "San Marino", "Sao Tome and Principe", "Saudi Arabia", "Senegal", "Scotland",
    "Serbia", "Seychelles", "Sierra Leone", "Singapore", "Slovakia", "Slovenia", "Solomon Islands", "Somalia",
    "South Africa", "South Korea", "South Sudan", "Spain", "Sri Lanka", "Sudan", "Suriname", "Sweden", "Switzerland",
    "Syria", "Tajikistan", "Taiwan", "Tanzania", "Thailand", "Timor-Leste", "Togo", "Tonga", "Trinidad and Tobago",
    "Tunisia", "Turkey", "Turkmenistan", "Tuvalu", "Uganda", "Ukraine", "United Arab Emirates", "United Kingdom",
    "United States", "Uruguay", "USSR", "Uzbekistan", "Vanuatu", "Venezuela", "Vietnam", "Yemen", "Zambia", "Zimbabwe"
]

#definimos una función para mostrar solo el país en la columna
def obtener_pais(location):
    for pais in lista_paises:
        if pais.lower() in location.lower():
            return pais
    return 'other'

# Aplicamos la función 'obtener_país' a la columna 'pais' y creamos la columna 'pais'
aa['pais'] = aa['pais'].apply(obtener_pais)

In [57]:
#vamos con la columna 'operadora'
# Reemplazamos los valores nulos por un valor vacío
aa['operadora'] = aa['operadora'].fillna('')

# Reemplazamos los valores de ? por un valor vacío
aa['operadora'] = aa['operadora'].str.replace('?', '')


In [58]:
# Reemplazamos los valores nulos por un valor vacío
aa['vuelo_numero'] = aa['vuelo_numero'].fillna('')

# Reemplazamos los valores de ? por un valor vacío
aa['vuelo_numero'] = aa['vuelo_numero'].str.replace('?', '')


In [59]:


# Reemplazamos los valores nulos por un valor vacío
aa['ruta'] = aa['ruta'].fillna('')

# Reemplazamos los valores de ? por un valor vacío
aa['ruta'] = aa['ruta'].str.replace('?', '')



In [60]:
# Reemplazamos los valores nulos por un valor vacío
aa['tipo_aeronave'] = aa['tipo_aeronave'].fillna('')

# Reemplazamos los valores de ? por un valor vacío
aa['tipo_aeronave'] = aa['tipo_aeronave'].str.replace('?', '')

In [61]:
#tomamos  una lista de marcas de aeronaves 
marcas_aeronaves_historicas = ['Wright Brothers', 'Boeing', 'Airbus', 'Lockheed Martin', 'McDonnell Douglas',
                              'Northrop Grumman', 'Convair', 'Grumman', 'Vickers', 'De Havilland', 'Hawker',
                              'Avro', 'Fokker', 'Junkers', 'Douglas Aircraft Company', 'Cessna', 'Piper Aircraft',
                              'Beechcraft', 'Bell Aircraft Corporation', 'Martin Marietta', 'Fairchild Aircraft',
                              'Bristol Aeroplane Company', 'Handley Page', 'Curtiss-Wright Corporation', 'Saab',
                              'Aérospatiale', 'Tupolev', 'Ilyushin', 'Antonov', 'Yakovlev', 'Sud Aviation',
                              'Dornier', 'Breguet Aviation', 'Messerschmitt', 'Vought', 'Piaggio Aerospace',
                              'Sikorsky Aircraft Corporation', 'Short Brothers', 'Supermarine', 'Avro Canada',
                              'Harbin Aircraft Manufacturing Corporation', 'English Electric',
                              'Mitsubishi Heavy Industries', 'Aichi', 'Nakajima', 'Kawasaki Heavy Industries',
                              'Handley Page', 'Armstrong Whitworth Aircraft', 'Blackburn Aircraft',
                              'Sopwith Aviation Company']

In [62]:
# Función para asignar valores a la columna 'brand'
def assign_brand(aircraft_type):
    for brand in marcas_aeronaves_historicas:
        if any(word.lower() in aircraft_type.lower().split() for word in brand.split()):
            return brand
    return 'other'

# Aplicamos la función a la columna 'aircraft_type' para crear la columna 'brand'
aa['tipo_aeronave'] = aa['tipo_aeronave'].apply(assign_brand)

In [63]:
# Reemplazar los valores nulos por un valor vacío
aa['abordo'] = aa['abordo'].fillna('')

# Reemplazar los valores de ? por un valor vacío
aa['abordo'] = aa['abordo'].str.replace('?', '')

# Eliminar filas con valores que no son números válidos (NaN y cadenas no numéricas)
aa = aa[pd.to_numeric(aa['abordo'], errors='coerce').notna()]

# Convertir la columna 'a bordo' a tipo entero (int)
aa['abordo'] = aa['abordo'].astype(int)


/tmp/ipykernel_7586/81569395.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['abordo'] = aa['abordo'].astype(int)


In [64]:
# Reemplazamos los valores nulos por un valor vacío
aa['registro_aeronave'] = aa['registro_aeronave'] .fillna('')

# Reemplazamos los valores de ? por un valor vacío
aa['registro_aeronave']  = aa['registro_aeronave'] .str.replace('?', '')

# Convertimos la columna 'a bordo' a tipo numérico 
aa['registro_aeronave']  = pd.to_numeric(aa['registro_aeronave'] , errors='coerce')

/tmp/ipykernel_7586/3674777899.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['registro_aeronave'] = aa['registro_aeronave'] .fillna('')
/tmp/ipykernel_7586/3674777899.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['registro_aeronave']  = aa['registro_aeronave'] .str.replace('?', '')
/tmp/ipykernel_7586/3674777899.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [65]:
# Reemplazamos los valores nulos por un valor vacío
aa['bajas'] = aa['bajas'].fillna('')

# Reemplazamos los valores de ? por un valor vacío
aa['bajas'] = aa['bajas'].str.replace('?', '')

# Convertimos la columna 'total_abord' a tipo numérico 
aa['bajas'] = pd.to_numeric(aa['bajas'], errors='coerce')

/tmp/ipykernel_7586/2989187705.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['bajas'] = aa['bajas'].fillna('')
/tmp/ipykernel_7586/2989187705.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['bajas'] = aa['bajas'].str.replace('?', '')
/tmp/ipykernel_7586/2989187705.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [66]:
# Reemplazar los valores nulos por un valor vacío
aa['tripulantes'] = aa['tripulantes'].fillna('')

# Reemplazar los valores de ? por un valor vacío
aa['tripulantes'] = aa['tripulantes'].str.replace('?', '')

# Utilizar pd.to_numeric con errors='coerce' para convertir a números y mantener NaN en lugar de eliminar filas
aa['tripulantes'] = pd.to_numeric(aa['tripulantes'], errors='coerce')

# Convertir la columna 'tripulantes' a tipo entero (int) manteniendo NaN
aa['tripulantes'] = aa['tripulantes'].fillna(pd.NA).astype('Int64')


/tmp/ipykernel_7586/868821141.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['tripulantes'] = aa['tripulantes'].fillna('')
/tmp/ipykernel_7586/868821141.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['tripulantes'] = aa['tripulantes'].str.replace('?', '')
/tmp/ipykernel_7586/868821141.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [67]:
# Reemplazar los valores nulos por un valor vacío
aa['bajas_pasajeros'] = aa['bajas_pasajeros'].fillna('')

# Reemplazar los valores de ? por un valor vacío
aa['bajas_pasajeros'] = aa['bajas_pasajeros'].str.replace('?', '')

# Utilizar pd.to_numeric con errors='coerce' para convertir a números y mantener NaN en lugar de eliminar filas
aa['bajas_pasajeros'] = pd.to_numeric(aa['bajas_pasajeros'], errors='coerce')

# Convertir la columna 'tripulantes' a tipo entero (int) manteniendo NaN
aa['bajas_pasajeros'] = aa['bajas_pasajeros'].fillna(pd.NA).astype('Int64')


/tmp/ipykernel_7586/2228817691.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['bajas_pasajeros'] = aa['bajas_pasajeros'].fillna('')
/tmp/ipykernel_7586/2228817691.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['bajas_pasajeros'] = aa['bajas_pasajeros'].str.replace('?', '')
/tmp/ipykernel_7586/2228817691.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [68]:
# Reemplazar los valores nulos por un valor vacío
aa['bajas_tripulacion'] = aa['bajas_tripulacion'].fillna('')

# Reemplazar los valores de ? por un valor vacío
aa['bajas_tripulacion'] = aa['bajas_tripulacion'].str.replace('?', '')

# Utilizar pd.to_numeric con errors='coerce' para convertir a números y mantener NaN en lugar de eliminar filas
aa['bajas_tripulacion'] = pd.to_numeric(aa['bajas_tripulacion'], errors='coerce')

# Convertir la columna 'tripulantes' a tipo entero (int) manteniendo NaN
aa['bajas_tripulacion'] = aa['bajas_tripulacion'].fillna(pd.NA).astype('Int64')


/tmp/ipykernel_7586/3341154712.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['bajas_tripulacion'] = aa['bajas_tripulacion'].fillna('')
/tmp/ipykernel_7586/3341154712.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['bajas_tripulacion'] = aa['bajas_tripulacion'].str.replace('?', '')
/tmp/ipykernel_7586/3341154712.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [69]:
# Reemplazar los valores nulos por un valor vacío
aa['bajas_tierra'] = aa['bajas_tierra'].fillna('')

# Reemplazar los valores de ? por un valor vacío
aa['bajas_tierra'] = aa['bajas_tierra'].str.replace('?', '')

# Utilizar pd.to_numeric con errors='coerce' para convertir a números y mantener NaN en lugar de eliminar filas
aa['bajas_tierra'] = pd.to_numeric(aa['bajas_tierra'], errors='coerce')

# Convertir la columna 'tripulantes' a tipo entero (int) manteniendo NaN
aa['bajas_tierra'] = aa['bajas_tierra'].fillna(pd.NA).astype('Int64')

/tmp/ipykernel_7586/1730188941.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['bajas_tierra'] = aa['bajas_tierra'].fillna('')
/tmp/ipykernel_7586/1730188941.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['bajas_tierra'] = aa['bajas_tierra'].str.replace('?', '')
/tmp/ipykernel_7586/1730188941.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [70]:
# Reemplazar los valores nulos por un valor vacío
aa['pasajeros'] = aa['pasajeros'].fillna('')

# Reemplazar los valores de ? por un valor vacío
aa['pasajeros'] = aa['pasajeros'].str.replace('?', '')

# Utilizar pd.to_numeric con errors='coerce' para convertir a números y mantener NaN en lugar de eliminar filas
aa['pasajeros'] = pd.to_numeric(aa['pasajeros'], errors='coerce')

# Convertir la columna 'tripulantes' a tipo entero (int) manteniendo NaN
aa['pasajeros'] = aa['pasajeros'].fillna(pd.NA).astype('Int64')

/tmp/ipykernel_7586/173493017.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['pasajeros'] = aa['pasajeros'].fillna('')
/tmp/ipykernel_7586/173493017.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['pasajeros'] = aa['pasajeros'].str.replace('?', '')
/tmp/ipykernel_7586/173493017.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

In [71]:
def limpiar_descripcion(texto):
    # Utiliza una expresión regular para eliminar caracteres especiales, incluido '\r'
    texto_limpio = re.sub(r'[^a-zA-Z0-9\s]', '', re.sub(r'[\r]', ' ', texto))
    return texto_limpio

# debido a '\r' tuve algunos inconvenientes.
aa['Descripcion'] = aa['Descripcion'].apply(limpiar_descripcion)
aa['operadora'] = aa['operadora'].apply(limpiar_descripcion)
aa['ruta'] = aa['ruta'].apply(limpiar_descripcion)



/tmp/ipykernel_7586/1399452123.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['Descripcion'] = aa['Descripcion'].apply(limpiar_descripcion)
/tmp/ipykernel_7586/1399452123.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['operadora'] = aa['operadora'].apply(limpiar_descripcion)
/tmp/ipykernel_7586/1399452123.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [72]:
# guardo en csv. 
aa.to_csv('avis.csv',index=False,sep='|',encoding='utf-8')

In [73]:
aa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4991 entries, 0 to 5007
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   fecha              4991 non-null   datetime64[ns]
 1   hora               4991 non-null   object        
 2   pais               4991 non-null   object        
 3   operadora          4991 non-null   object        
 4   vuelo_numero       4991 non-null   object        
 5   ruta               4991 non-null   object        
 6   tipo_aeronave      4991 non-null   object        
 7   registro_aeronave  189 non-null    float64       
 8   abordo             4991 non-null   int64         
 9   pasajeros          4787 non-null   Int64         
 10  tripulantes        4789 non-null   Int64         
 11  bajas              4991 non-null   int64         
 12  bajas_pasajeros    4770 non-null   Int64         
 13  bajas_tripulacion  4771 non-null   Int64         
 14  bajas_tierra 